<a href="https://colab.research.google.com/github/CalvHobbes/pricecomp_agents/blob/main/llamaindex/llamaindex_workflow_price_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 requests

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py -o /dev/null
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py -o /dev/null


In [5]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

def test_functions():
  # Define the product query
  query = "iphone 16 pro max"

  # Fetch prices from Croma
  croma_prices = fetch_prices_from_croma(query)
  print("Croma Prices:")
  print(json.dumps(croma_prices, indent=4))

  # Fetch prices from Reliance Digital
  reliance_prices = fetch_prices_from_reliance(query)
  print("Reliance Digital Prices:")
  print(json.dumps(reliance_prices, indent=4))


# test_functions()





In [ ]:
!pip install llama-index llama-index-llms-openai

In [9]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [12]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event


class InputEvent(Event):
    input: list[ChatMessage]


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

In [14]:
from typing import Any, List

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


class FuncationCallingAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: FunctionCallingLLM | None = None,
        tools: List[BaseTool] | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()
        assert self.llm.metadata.is_function_calling_model

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.sources = []

    @step
    async def prepare_chat_history(self, ev: StartEvent) -> InputEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # get chat history
        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

    @step
    async def handle_llm_input(
        self, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input

        response = await self.llm.achat_with_tools(
            self.tools, chat_history=chat_history
        )
        self.memory.put(response.message)

        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )

        if not tool_calls:
            return StopEvent(
                result={"response": response, "sources": [*self.sources]}
            )
        else:
            return ToolCallEvent(tool_calls=tool_calls)

    @step
    async def handle_tool_calls(self, ev: ToolCallEvent) -> InputEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        tool_msgs = []

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            additional_kwargs = {
                "tool_call_id": tool_call.tool_id,
                "name": tool.metadata.get_name(),
            }
            if not tool:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Tool {tool_call.tool_name} does not exist",
                        additional_kwargs=additional_kwargs,
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=tool_output.content,
                        additional_kwargs=additional_kwargs,
                    )
                )
            except Exception as e:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Encountered error in tool call: {e}",
                        additional_kwargs=additional_kwargs,
                    )
                )

        for msg in tool_msgs:
            self.memory.put(msg)

        chat_history = self.memory.get()
        return InputEvent(input=chat_history)




In [15]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

# model = OpenAI(model="gpt-3.5-turbo")
model = OpenAI(model="gpt-4o-mini")



In [16]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [17]:
from llama_index.core.tools import FunctionTool

croma_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_croma,
    name="croma_prices",
    description="Fetches product prices from Chroma")

reliance_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_reliance,
    name="reliance_prices",
    description="Fetches product prices from Reliance Digital")



In [21]:

agent = FuncationCallingAgent(
    llm=model, tools=[croma_tool, reliance_tool], timeout=120, verbose=True
)


In [27]:
from IPython.display import display, Markdown


response = await agent.run(input="What is the best price for LG 9kg  Front Load Washing Machine?")
display(Markdown(response.text))

Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
Step handle_tool_calls produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
Step handle_tool_calls produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event StopEvent


AttributeError: 'dict' object has no attribute 'response'

In [55]:
print(response['response'])


assistant: Here are the best prices for the LG 9kg Front Load Washing Machine from Croma and Reliance Digital:

### Croma
1. **[LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M)](https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606)** - ₹43,990.00
2. **[LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B)](https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332)** - ₹45,990.00
3. **[LG 9 Kg 5 Star Inverter Fully Automatic Front Load Washing Machine (FHP1209Z9B)](https://www.croma.com/lg-9-kg-5-star-inverter-fully-automatic-front-load-washing-machine-fhp1209z9b-ablqeil-in-built-heater-steam-plus-black-/p/270905)** - ₹44,990.00

### Reliance Digital
1. **[LG 9 Kg Fully Automatic Front Loading Washing Machine (FHB1209Z4B)](https://www.reliancedigital.in/lg-9-kg-fu